In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 42.7 MB/s eta 0:00:00


In [ ]:
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
from transformers import ElectraPreTrainedModel ,ElectraModel
from transformers import  ElectraTokenizerFast as ElectraTokenizer 
from transformers.models.electra.modeling_electra import ElectraClassificationHead

In [ ]:
class ElectraClassifier(ElectraPreTrainedModel):

  def __init__(self,config):
    super().__init__(config)
    self.n_classes = config.num_labels
    self.config = config
    self.electra = ElectraModel(config)
    self.classifier = ElectraClassificationHead(config)
    self.post_init()

  def forward(self, input_ids=None,attention_mask=None):
    discriminator_hidden_states = self.electra(input_ids, attention_mask)
    sequence_output = discriminator_hidden_states[0]
    logits = self.classifier(sequence_output)
    return logits

In [ ]:
emotions_categories = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
       'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
       'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
       'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
       'relief', 'remorse', 'sadness', 'surprise', 'neutral']

In [ ]:
emotions_categories

['admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']

In [ ]:
MODEL_NAME = "google/electra-base-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
loaded_model = ElectraClassifier.from_pretrained("/content/drive/MyDrive/EmotionClassifier_ELECTRA")

In [ ]:
def classifyEmotions_new(text, trainer_model, tokenizer, emotions_categories, threshold=0.9):
    text_encoding = tokenizer(
        text,
        max_length = 64,
        truncation =True,
        padding = "max_length",
        add_special_tokens=True,
        return_token_type_ids=False,
        return_attention_mask = True,
        return_tensors="pt"
    )
    with torch.no_grad():
      pred = trainer_model(text_encoding["input_ids"],text_encoding["attention_mask"])
      pred= pred.flatten().numpy()
    

    pred_labels = []
    for i, label_name in enumerate(emotions_categories):
      label_prob = pred[i]
      if label_prob > threshold :
        pred_labels.append(label_name)

    
    return pred_labels

In [ ]:
classifyEmotions_new("im very sad",loaded_model, tokenizer, emotions_categories, threshold=1)

['disappointment', 'remorse', 'sadness']